In [289]:
import base64
import os
from datetime import datetime, timedelta
from urllib.parse import urlencode

import requests

import base64
import os
from abc import ABC, abstractmethod, abstractproperty
from datetime import datetime, timedelta
from urllib.parse import urlencode

import requests

class AuthManager:
    TOKEN_URL = "https://accounts.spotify.com/api/token"
    AUTH_URL = "https://accounts.spotify.com/authorize"
    
    def __init__(self, client_id, client_secret, redirect_uri=None, scope=None, state=None):
        self._client_id = client_id
        self._client_secret = client_secret
        self._redirect_uri = redirect_uri
        self._scope = scope
        self._state = state
        
    @property
    def client_credentials(self):
        client_credentials = f"{self._client_id}:{self._client_secret}"
        self._client_credentials = base64.b64encode(client_credentials.encode())
        return self._client_credentials.decode()
    
    @property
    def redirect_uri(self):
        return self._redirect_uri
    
    # Add setter for scope and validate
    @property
    def scope(self):
        return self._scope
    
    @property
    def grant_type(self):
        if not self._redirect_uri:
            return {"grant_type": "client_credentials"}
        return {"grant_type": "authorization_code"}
        
        
    # Group this as 'auth'?
    @property
    def basic_auth(self):
        return {"Authorization": f"Basic {self.client_credentials}"}
    
    # ""
    @property
    def bearer_auth(self):
        pass
    
    @property
    def content_type(self):
        return {'content_type':'application/x-www-form-urlencoded'}
    
    @property
    def token_data(self):
        return self.grant_type
        
    # Change this to just be auth when merging the basic and bearer auth tokens
    @property
    def token_headers(self):
        return self.basic_auth | self.content_type
    
    def authorise(self, data, headers):
        pass
    
    def request_token(self):
        url = self.TOKEN_URL
        data = self.token_data
        headers = self.token_headers
        r = requests.post(url, data=data, headers=headers)
        if r.status_code not in range(200, 299):
            raise Exception('Could not authenticate user, please try again')
        return r.json()
        
    
class Client:
    BASE_URL = 'https://api.spotify.com/v1'
    
    def __init__(self, client_id, client_secret, redirect_uri=None, scope=None, state=None):
        self._auth_manager = AuthManager(client_id, client_secret, redirect_uri, scope)
        
    @property
    def client_credentials(self):
        client_credentials = f"{self._client_id}:{self._client_secret}"
        self._client_credentials = base64.b64encode(client_credentials.encode())
        return self._client_credentials.decode()
    
    @property
    def token_info(self):
        return self._token_info
    
    @token_info.setter
    def token_info(self, val):
        # Assert if this is a valid token?
        self._token_info = val

    @property
    def auth_manager(self):
        return self._auth_manager
    
    @auth_manager.setter
    def auth_manager(self, val):
        if isinstance(val, AuthManager):
            self._auth_manager = val

    def authorise(self):
        self.token_info = self.auth_manager.request_token()
    
    @property
    def resource_headers(self):
        access_token = self.token_info.get('access_token')
        return {'Authorization':f'Bearer {access_token}'}
    
    # TODO  
    def get_resource(self, lookup_id, resource_type='albums'):
        endpoint = f'{self.BASE_URL}/{resource_type}/{lookup_id}'
        headers = self.resource_headers
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range (200,299):
            return {}
        return r.json()
    
    def get_album(self, _id):
        return self.get_resource(_id, 'albums')
    
    
    def get_artist(self, _id):
        return self.get_resource(_id, 'artists')
    
    
    def search(self, query, search_type='track'):
        endpoint = f'{self.BASE_URL}/search'
        headers = self.resource_headers
        data = urlencode({'q':query,'type':search_type.lower()})
        url = f'{endpoint}?{data}'
        print(url)
        r = requests.get(url, headers=headers)
        if r.status_code not in range (200,299):
            return {}
        return r.json()
        
    